In [1]:
from cnfgen import RandomKCNF
import numpy as np
from scipy.sparse import csr_matrix,save_npz,load_npz
from collections import defaultdict
import os

In [20]:
def create_directed_edges(adj_matrix, edges):
    for edge in edges:
        i, j = edge
        adj_matrix[i, j] += 1
        adj_matrix[j, i] += 1
def formula_to_sparse(formula):
    # SAT-MIS
    try: 
        clauses=formula.clauses()
    except:
        clauses=formula.clauses

    m = len(clauses)
    adj_matrix = np.zeros((3 * m, 3 * m))
    literal_to_node = defaultdict(list)

    i = 0

    # Clause clusters
    for clause in clauses:
        create_directed_edges(adj_matrix, [(i, i + 1), (i + 1, i + 2), (i + 2, i)])

        for literal in clause:
            literal_to_node[literal].append(i)
        i += 3

    # Conflict links
    for literal in literal_to_node:
        if -literal in literal_to_node:
            for node in literal_to_node[literal]:
                for neighbour in literal_to_node[-literal]:
                    adj_matrix[node, neighbour] = 1

    # MIS-MC
    N = adj_matrix.shape[0]
    number_of_nodes = N + 1 + np.count_nonzero(adj_matrix)
    adj_matrix_mc = np.zeros((number_of_nodes, number_of_nodes))
    idx = N + 1

    for i in range(adj_matrix.shape[0]):
        for j in range(i):
            if adj_matrix[i][j] > 0:
                edges = [(N, i), (N, j), (N, idx), (N, idx + 1),
                            (i, idx), (idx, idx + 1), (idx + 1, j)]
                create_directed_edges(adj_matrix_mc, edges)
                idx += 2
    sparse_matrix = csr_matrix(adj_matrix_mc)
    return sparse_matrix
    

In [2]:
def mk_dir(export_dir, quite=False):
    if not os.path.exists(export_dir):
            try:
                os.makedirs(export_dir)
                print('created dir: ', export_dir)
            except OSError as exc: # Guard against race condition
                 if exc.errno != exc.errno.EEXIST:
                    raise
            except Exception:
                pass
    else:
        print('dir already exists: ', export_dir)

In [17]:
def create_instance(k, n, m, number_of_instances, folder):
    mk_dir(folder)
    for instance_no in range(number_of_instances):
        formula = RandomKCNF(k=k, n=n, m=m, seed=None, planted_assignments=None)
        sparse_matrix=formula_to_sparse(formula)
        save_npz(f'{folder}/k{k}_n{n}_m{m}_instance_{instance_no}', sparse_matrix)

In [5]:
create_instance(k=3, n=20, m=91, number_of_instances=4000, folder='data/training/Uniform Random-3-SAT/')
create_instance(k=3, n=20, m=91, number_of_instances=50, folder='data/validation/Uniform Random-3-SAT/')
# create_instance(k=3, n=20, m=91, number_of_instances=100, folder='data/testing/Uniform Random-3-SAT/')

dir already exists:  data/training/Uniform Random-3-SAT/
dir already exists:  data/validation/Uniform Random-3-SAT/
dir already exists:  data/testing/Uniform Random-3-SAT/


# SATLIB Dataset

In [18]:
class CNF:
    def __init__(self, n_variables, clauses, occur_list, comments):
        self.n_variables = n_variables
        self.clauses = clauses
        self.occur_list = occur_list
        self.comments = comments

    def __str__(self):
        return f"""Number of variables: {self.n_variables}
                Clauses: {str(self.clauses)}
                Comments: {str(self.comments)}"""

    def to_file(self, filename):
        with open(filename, 'w') as f:
            f.write(self.to_string())

    def to_string(self):
        string = f'p cnf {self.n_variables} {len(self.clauses)}\n'
        for clause in self.clauses:
            string += ' '.join(str(literal) for literal in clause) + ' 0\n'
        return string

    @classmethod
    def from_file(cls, filename):
        with open(filename) as f:
            return cls.from_string(f.read())

    @classmethod
    def from_string(cls, string):
        n_variables, clauses, occur_list, comments = CNF.parse_dimacs(string)
        return cls(n_variables, clauses, occur_list, comments)

    @staticmethod
    def parse_dimacs(string):
        n_variables = 0
        clauses = []
        comments = []
        for line in string.splitlines():
            line = line.strip()
            if not line:
                continue
            elif line[0] == 'c':
                comments.append(line)
            elif line.startswith('p cnf'):
                tokens = line.split()
                n_variables, n_remaining_clauses = int(tokens[2]), int(tokens[3])
                occur_list = [[] for _ in range(n_variables * 2 + 1)]
            elif n_remaining_clauses > 0:
                clause = []
                clause_index = len(clauses)
                for literal in line.split()[:-1]:
                    literal = int(literal)
                    clause.append(literal)
                    occur_list[literal].append(clause_index)
                clauses.append(clause)
                n_remaining_clauses -= 1
            else:
                break
        return n_variables, clauses, occur_list, comments

In [23]:
import os
k=3
n=20
m=91

load_folder='SATLIB'
files=os.listdir(load_folder)

for instance_no,file in enumerate(files):
    formula=CNF.from_file(os.path.join(load_folder,file))
    sparse_matrix=formula_to_sparse(formula)
    save_npz(f'data/testing/Uniform Random-3-SAT/k{k}_n{n}_m{m}_instance_{instance_no}', sparse_matrix)
    